In [ ]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# ============================================================
# 0) 設定（ここだけ）
# ============================================================
FEAT_TR_CSV = r"features_train_s1.csv"
FEAT_TE_CSV = r"features_test_s1.csv"

# 特徴量CSVの「時刻」列（無ければ None）
TIME_COL  = "center_idx"   # 例: "center_idx" / 無ければ None
# 特徴量CSVの「ラベル」列（無ければ None）
LABEL_COL = "win_label"    # 例: "win_label" / 無ければ None

# PCAに使う列（特徴量列）を指定
PCA_COLS  = ['s1__psd_peak_freq', 's1__psd_peak_power', 's1__bandpower_0_0.01', 's1__bandpower_0.01_0.03', 's1__bandpower_0.03_0.5', 's1__phase_lag', 's1__xcorr_score', 's1__corrcoef_aligned', 's1__mse', 's1__rmse', 's1__nrmse', 's1__mae', 's1__mean', 's1__median', 's1__has_hit', 's1__peak_amplitude', 's1__hit_samples', 's1__counts', 's1__duration_samples', 's1__rise_time_samples', 's1__signal_strength_l1', 's1__energy_l2'
]  #

# PCA 次元数（累積寄与率）
VAR_RATIO = 0.95
# 閾値（学習スコアの分位点）
ALPHA = 0.995


# ============================================================
# 1) 読み込み（必要最低限の堅牢版）
# ============================================================
def read_csv_robust(path: str) -> pd.DataFrame:
    for enc in ("utf-8-sig", "utf-8", "cp932", "shift_jis"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    return pd.read_csv(path, encoding_errors="replace")

df_tr = read_csv_robust(FEAT_TR_CSV)
df_te = read_csv_robust(FEAT_TE_CSV)

# まず DataFrame とカラムを確認（ここを見て TIME_COL / LABEL_COL / PCA_COLS を決める）
display(df_tr.head())
print("TRAIN columns:", list(df_tr.columns))
display(df_te.head())
print("TEST columns :", list(df_te.columns))


# ============================================================
# 2) PCA学習（train）
# ============================================================
Xtr = df_tr[PCA_COLS].apply(pd.to_numeric, errors="coerce").to_numpy(float)
Xte = df_te[PCA_COLS].apply(pd.to_numeric, errors="coerce").to_numpy(float)

col_mean = np.nanmean(Xtr, axis=0)
bad = ~np.isfinite(Xtr)
if bad.any():
    Xtr[bad] = col_mean[np.where(bad)[1]]
bad = ~np.isfinite(Xte)
if bad.any():
    Xte[bad] = col_mean[np.where(bad)[1]]

scaler = StandardScaler()
Xtr_s = scaler.fit_transform(Xtr)
Xte_s = scaler.transform(Xte)

pca0 = PCA(svd_solver="full").fit(Xtr_s)
k = int(np.searchsorted(np.cumsum(pca0.explained_variance_ratio_), VAR_RATIO) + 1)
k = max(1, min(k, Xtr_s.shape[1]))
pca = PCA(n_components=k, svd_solver="full").fit(Xtr_s)

print("PCA components:", pca.n_components_)
print("EVR sum:", float(np.sum(pca.explained_variance_ratio_)))


# ============================================================
# 3) MSPCスコア（T2, Q） + 閾値（分位点）
# ============================================================
def mspc_scores(Xs, pca):
    T = pca.transform(Xs)
    Xhat = pca.inverse_transform(T)
    E = Xs - Xhat
    lam = np.maximum(pca.explained_variance_, 1e-12)
    T2 = np.sum((T**2) / lam[None, :], axis=1)
    Q  = np.sum(E**2, axis=1)
    return T2, Q, T

T2_tr, Q_tr, T_tr = mspc_scores(Xtr_s, pca)
T2_te, Q_te, T_te = mspc_scores(Xte_s, pca)

T2_lim = float(np.quantile(T2_tr, ALPHA))
Q_lim  = float(np.quantile(Q_tr,  ALPHA))
print("T2_limit:", T2_lim, "Q_limit:", Q_lim)

def build_out(df_src, T2, Q, T2_lim, Q_lim):
    out = pd.DataFrame({"T2": T2, "Q": Q})
    out["T2_limit"] = T2_lim
    out["Q_limit"]  = Q_lim
    out["is_alarm"] = ((out["T2"] > T2_lim) | (out["Q"] > Q_lim)).astype(int)
    if TIME_COL is not None:
        out[TIME_COL] = df_src[TIME_COL].to_numpy()
    if LABEL_COL is not None:
        out[LABEL_COL] = df_src[LABEL_COL].to_numpy()
    return out

df_mspc_tr = build_out(df_tr, T2_tr, Q_tr, T2_lim, Q_lim)
df_mspc_te = build_out(df_te, T2_te, Q_te, T2_lim, Q_lim)

display(df_mspc_te.head())


# ============================================================
# 4) スコア波形プロット（背景：ラベル区間）
# ============================================================
def _get_x(df):
    if TIME_COL is None:
        return np.arange(len(df), dtype=float), "index"
    x = pd.to_numeric(df[TIME_COL], errors="coerce").to_numpy(float)
    if np.isnan(x).all():
        return np.arange(len(df), dtype=float), "index"
    return x, TIME_COL

def _label_segments(x, labels):
    lab = labels.astype(object).copy()
    for i in range(len(lab)):
        if pd.isna(lab[i]):
            lab[i] = "NaN"
    segs, s = [], 0
    for i in range(1, len(lab)):
        if lab[i] != lab[i-1]:
            segs.append((x[s], x[i-1], lab[i-1])); s = i
    segs.append((x[s], x[-1], lab[-1]))
    return segs

def plot_scores(df, title):
    x, xlab = _get_x(df)
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12, 6))

    if (LABEL_COL is not None) and (LABEL_COL in df.columns):
        segs = _label_segments(x, df[LABEL_COL].to_numpy())
        uniq = []
        for _, _, lb in segs:
            if lb not in uniq:
                uniq.append(lb)
        cmap = plt.get_cmap("tab10")
        col = {lb: cmap(i % 10) for i, lb in enumerate(uniq)}
        for xs, xe, lb in segs:
            ax1.axvspan(xs, xe, alpha=0.18, color=col[lb])
            ax2.axvspan(xs, xe, alpha=0.18, color=col[lb])

    ax1.plot(x, df["T2"].to_numpy(), lw=0.9)
    ax1.plot(x, df["T2_limit"].to_numpy(), lw=1.2)
    ax2.plot(x, df["Q"].to_numpy(), lw=0.9)
    ax2.plot(x, df["Q_limit"].to_numpy(), lw=1.2)

    ax1.set_ylabel("T2"); ax2.set_ylabel("Q")
    ax2.set_xlabel(xlab)
    ax1.grid(True); ax2.grid(True)
    fig.suptitle(title); fig.tight_layout()
    plt.show()

plot_scores(df_mspc_tr, "MSPC TRAIN (T2/Q)")
plot_scores(df_mspc_te, "MSPC TEST (T2/Q)")


# ============================================================
# 5) 寄与率（explained variance ratio）
# ============================================================
evr = pca.explained_variance_ratio_
plt.figure(figsize=(12, 3))
plt.bar(np.arange(1, len(evr)+1), evr)
plt.xticks(np.arange(1, len(evr)+1), [f"PC{i}" for i in range(1, len(evr)+1)], rotation=45, ha="right")
plt.ylabel("explained variance ratio")
plt.title("PCA explained variance ratio")
plt.grid(True)
plt.tight_layout()
plt.show()


# ============================================================
# 6) 散布図（seaborn pairplot：TEST、ラベルで色分け）
# ============================================================
k2 = min(4, T_te.shape[1])
pc_cols = [f"PC{i}" for i in range(1, k2+1)]
df_pc = pd.DataFrame(T_te[:, :k2], columns=pc_cols)

if (LABEL_COL is not None) and (LABEL_COL in df_mspc_te.columns):
    hue = LABEL_COL
    df_pc[hue] = (
        df_mspc_te[hue]
        .astype(str)
        .str.replace(r"\.0$", "", regex=True)
        .fillna("NaN")
    )
else:
    hue = "is_alarm"
    df_pc[hue] = df_mspc_te["is_alarm"].astype(int).astype(str)

sns.pairplot(df_pc, vars=pc_cols, hue=hue, diag_kind="hist", plot_kws=dict(s=12, alpha=0.7))
plt.show()

# ============================================================
# 7) 負荷量（loading）と「主成分ごとの寄与（loading^2をPCごとに正規化）」 
#    - loading: 符号つき（どっち向きに効くか）
#    - 寄与: 0〜1（そのPCを作っている割合、合計=1）
# ============================================================

# PC名
pc_names = [f"PC{i}" for i in range(1, pca.n_components_ + 1)]

# 負荷量（符号つき）
loadings = pca.components_.T                       # (n_features, n_components)
df_load = pd.DataFrame(loadings, index=PCA_COLS, columns=pc_names)

# 主成分ごとの寄与（loading^2 を各PC列で合計1に正規化）
df_contrib = (df_load ** 2)
df_contrib = df_contrib.div(df_contrib.sum(axis=0) + 1e-12, axis=1)

# --- ヒートマップは見やすいように上位だけ
K_PC = min(10, df_contrib.shape[1])   # 表示するPC数
TOP_M = 30                             # 表示する特徴量数

idx_top = (
    df_contrib.iloc[:, :K_PC]
    .sum(axis=1)
    .sort_values(ascending=False)
    .head(TOP_M)
    .index
)

plt.figure(figsize=(12, 0.35 * TOP_M + 2))
sns.heatmap(df_contrib.loc[idx_top, pc_names[:K_PC]], cmap="viridis")
plt.title("Feature contribution to PCs (normalized loading^2; each PC sums to 1)")
plt.xlabel("PC"); plt.ylabel("feature")
plt.tight_layout()
plt.show()

# --- 参考：符号つき負荷量（上位特徴量だけ）
plt.figure(figsize=(12, 0.35 * TOP_M + 2))
sns.heatmap(df_load.loc[idx_top, pc_names[:K_PC]], center=0, cmap="coolwarm")
plt.title("Loadings (signed) for top features")
plt.xlabel("PC"); plt.ylabel("feature")
plt.tight_layout()
plt.show()

# --- 各PCで「寄与が大きい特徴量トップ」を表で確認（符号も一緒に）
TOP_EACH = 10
for pc in pc_names[:min(6, len(pc_names))]:
    s = df_contrib[pc].sort_values(ascending=False).head(TOP_EACH)
    tmp = pd.DataFrame({
        "contrib": s,
        "loading": df_load.loc[s.index, pc]
    })
    print(f"\n=== {pc} top{TOP_EACH} features ===")
    display(tmp)



,win_i,start_idx,end_idx,center_idx,center_time,win_label,s1__psd_peak_freq,s1__psd_peak_power,s1__bandpower_0_0.01,s1__bandpower_0.01_0.03,...,s1__mean,s1__median,s1__has_hit,s1__peak_amplitude,s1__hit_samples,s1__counts,s1__duration_samples,s1__rise_time_samples,s1__signal_strength_l1,s1__energy_l2
0,0.0,0.0,255.0,128.0,1.28,0.0,0.003906,1.587733,0.010139,0.003301,...,-0.023815,-0.025211,0,0.437334,0,0,0,0,34.214193,6.951679
1,1.0,256.0,511.0,384.0,3.84,0.0,0.003906,1.300237,0.007989,0.003154,...,-0.000721,0.001087,0,0.440669,0,0,0,0,31.854413,6.263044
2,2.0,512.0,767.0,640.0,6.40,0.0,0.003906,1.776724,0.011484,0.003105,...,0.002677,-0.003005,0,0.412316,0,0,0,0,35.231192,7.242723
3,3.0,768.0,1023.0,896.0,8.96,0.0,0.003906,2.337662,0.015083,0.001812,...,0.000467,0.023405,0,0.445250,0,0,0,0,35.867120,7.677018
4,4.0,1024.0,1279.0,1152.0,11.52,0.0,0.003906,2.401367,0.015627,0.003654,...,-0.009961,-0.011410,0,0.494726,0,0,0,0,38.024510,8.563567


TRAIN columns: ['win_i', 'start_idx', 'end_idx', 'center_idx', 'center_time', 'win_label', 's1__psd_peak_freq', 's1__psd_peak_power', 's1__bandpower_0_0.01', 's1__bandpower_0.01_0.03', 's1__bandpower_0.03_0.5', 's1__phase_lag', 's1__xcorr_score', 's1__corrcoef_aligned', 's1__mse', 's1__rmse', 's1__nrmse', 's1__mae', 's1__mean', 's1__median', 's1__has_hit', 's1__peak_amplitude', 's1__hit_samples', 's1__counts', 's1__duration_samples', 's1__rise_time_samples', 's1__signal_strength_l1', 's1__energy_l2']


,win_i,start_idx,end_idx,center_idx,center_time,win_label,s1__psd_peak_freq,s1__psd_peak_power,s1__bandpower_0_0.01,s1__bandpower_0.01_0.03,...,s1__mean,s1__median,s1__has_hit,s1__peak_amplitude,s1__hit_samples,s1__counts,s1__duration_samples,s1__rise_time_samples,s1__signal_strength_l1,s1__energy_l2
0,0.0,0.0,255.0,128.0,1.28,0.0,0.003906,1.671537,0.010738,0.002605,...,0.004711,0.000296,0,0.575188,0,0,0,0,33.474025,6.827475
1,1.0,256.0,511.0,384.0,3.84,0.0,0.003906,1.500243,0.010064,0.001472,...,-0.000143,0.006924,0,0.509283,0,0,0,0,33.510339,6.755158
2,2.0,512.0,767.0,640.0,6.40,0.0,0.003906,1.757092,0.011529,0.001850,...,-0.005382,-0.013546,0,0.479643,0,0,0,0,35.546819,7.493408
3,3.0,768.0,1023.0,896.0,8.96,0.0,0.003906,1.750349,0.011338,0.004135,...,-0.005792,-0.013649,0,0.414483,0,0,0,0,35.470968,7.373979
4,4.0,1024.0,1279.0,1152.0,11.52,0.0,0.003906,1.487513,0.010196,0.002719,...,-0.002004,0.007579,0,0.389316,0,0,0,0,33.018970,6.177149


TEST columns : ['win_i', 'start_idx', 'end_idx', 'center_idx', 'center_time', 'win_label', 's1__psd_peak_freq', 's1__psd_peak_power', 's1__bandpower_0_0.01', 's1__bandpower_0.01_0.03', 's1__bandpower_0.03_0.5', 's1__phase_lag', 's1__xcorr_score', 's1__corrcoef_aligned', 's1__mse', 's1__rmse', 's1__nrmse', 's1__mae', 's1__mean', 's1__median', 's1__has_hit', 's1__peak_amplitude', 's1__hit_samples', 's1__counts', 's1__duration_samples', 's1__rise_time_samples', 's1__signal_strength_l1', 's1__energy_l2']
PCA components: 8
EVR sum: 0.9518032708556814
T2_limit: 78.29032189707668 Q_limit: 3.4036673104637476


,T2,Q,T2_limit,Q_limit,is_alarm,center_idx,win_label
0,18.731297,1.406881,78.290322,3.403667,0,128.0,0.0
1,7.205817,0.893839,78.290322,3.403667,0,384.0,0.0
2,4.733337,0.566130,78.290322,3.403667,0,640.0,0.0
3,10.188904,0.836014,78.290322,3.403667,0,896.0,0.0
4,10.360926,0.739971,78.290322,3.403667,0,1152.0,0.0
